In [1]:
import os
import numpy as np
import torch as tc
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm.notebook import tqdm, trange
import pickle

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# these are the helper libraries imported.
from engine import train_one_epoch, evaluate
import utils
# import transforms as T
#import cv2
from WishDataSet import WishDataSet, get_transform
from wish_utils import *

In [13]:
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(weights=None)
model

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /home/wj1132075/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|███████████████████████████████████████| 21.1M/21.1M [00:00<00:00, 294MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
          )
        )
      )
      (2): InvertedResidual(
        (block):

In [3]:
model.transform

GeneralizedRCNNTransform(
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    Resize(min_size=(800,), max_size=1333, mode='bilinear')
)

In [4]:
stats = (np.array([1.47101333, 1.47101333, 1.47101333]), np.array([4.50738405, 4.50738405, 4.50738405]))

In [5]:
from torchvision.models.detection.faster_rcnn import GeneralizedRCNNTransform
GeneralizedRCNNTransform(min_size=800, max_size=1333, mode='bilinear', image_mean=[0,2,0], image_std=[1,1,1])

GeneralizedRCNNTransform(
    Normalize(mean=[0, 2, 0], std=[1, 1, 1])
    Resize(min_size=(800,), max_size=1333, mode='bilinear')
)

In [14]:
model = get_model(model=torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True),
                 num_classes=2,
                 stats=stats)

/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth" to /home/wj1132075/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth
100%|██████████████████████████████████████| 74.2M/74.2M [00:05<00:00, 13.1MB/s]


In [19]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[1.47101333 1.47101333 1.47101333], std=[4.50738405 4.50738405 4.50738405])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
          )
        )
      )
      (2): Inverted

In [8]:
# def show_children(model):
#     l = list(model.children())
#     for ch in l:
#         show_children(ch)
#         print(type(ch))
    
#         # break

# show_children(model)

In [15]:
freeze_to(model, fr=False)

freezing parmas=0 at GeneralizedRCNNTransform(
    Normalize(mean=[1.47101333 1.47101333 1.47101333], std=[4.50738405 4.50738405 4.50738405])
    Resize(min_size=(800,), max_size=1333, mode='bilinear')
)
freezing parmas=1 at Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
---
 Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False) True
freezing parmas=0 at Hardswish()
freezing parmas=1 at Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
---
 Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False) True
freezing parmas=0 at ReLU(inplace=True)
freezing parmas=1 at Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
---
 Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False) True
freezing parmas=1 at Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
---
 Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False) True
freezing parmas=0 a

In [18]:
[p.shape for p in model.parameters() if p.requires_grad]

[torch.Size([16, 3, 3, 3]),
 torch.Size([16, 1, 3, 3]),
 torch.Size([16, 16, 1, 1]),
 torch.Size([64, 16, 1, 1]),
 torch.Size([64, 1, 3, 3]),
 torch.Size([24, 64, 1, 1]),
 torch.Size([72, 24, 1, 1]),
 torch.Size([72, 1, 3, 3]),
 torch.Size([24, 72, 1, 1]),
 torch.Size([72, 24, 1, 1]),
 torch.Size([72, 1, 5, 5]),
 torch.Size([24, 72, 1, 1]),
 torch.Size([24]),
 torch.Size([72, 24, 1, 1]),
 torch.Size([72]),
 torch.Size([40, 72, 1, 1]),
 torch.Size([120, 40, 1, 1]),
 torch.Size([120, 1, 5, 5]),
 torch.Size([32, 120, 1, 1]),
 torch.Size([32]),
 torch.Size([120, 32, 1, 1]),
 torch.Size([120]),
 torch.Size([40, 120, 1, 1]),
 torch.Size([120, 40, 1, 1]),
 torch.Size([120, 1, 5, 5]),
 torch.Size([32, 120, 1, 1]),
 torch.Size([32]),
 torch.Size([120, 32, 1, 1]),
 torch.Size([120]),
 torch.Size([40, 120, 1, 1]),
 torch.Size([240, 40, 1, 1]),
 torch.Size([240, 1, 3, 3]),
 torch.Size([80, 240, 1, 1]),
 torch.Size([200, 80, 1, 1]),
 torch.Size([200, 1, 3, 3]),
 torch.Size([80, 200, 1, 1]),
 torch.

In [20]:
model = get_object_detection_model(2, True)
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(